In [ ]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
! pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.9 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda')

In [ ]:
path = '/content/drive/MyDrive/KUBIG/이상치 탐지 프로젝트/data/'

In [ ]:
train_png = sorted(glob(path + '/train/train/*.png'))
test_png = sorted(glob(path + '/test/test/*.png'))

In [ ]:
len(train_png), len(test_png)

(4277, 2154)

In [ ]:
train_y = pd.read_csv(path +"train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (300, 300),interpolation = cv2.INTER_AREA)
    return img

In [ ]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [24:01<00:00,  1.49it/s]


In [ ]:
train_imgs[0].shape

(300, 300, 3)

In [ ]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",meanR, meanG, meanB)
print("train 표준편차",stdR, stdG, stdB)

train 평균 0.43303983566148774 0.4034587704308856 0.3941522437234577
train 표준편차 0.18081092522063152 0.17336374759777612 0.16260836048470312


In [ ]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균",meanR, meanG, meanB)
print("test 표준편차",stdR, stdG, stdB)

test 평균 0.41825790665590495 0.3931025338479558 0.38663352463775147
test 표준편차 0.19437405223049778 0.18947139729295545 0.17999943333552182


In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433039, 0.403458, 0.394152],
                                     std = [0.180810, 0.173363, 0.162608]),
                transforms.RandomAffine((-180, 180)),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418257, 0.393102, 0.386633],
                                     std = [0.194374, 0.189471, 0.179999])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=88, drop_path_rate = 0.2)
        if self.mode == 'test':
          self.model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [ ]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

### 모델 학습

사전 학습 모델의 성능 파악을 할 때 Fold 학습은 실행 시간이 오래걸려서 fold를 나누지 않은 데이터에 대해서 학습을 진행하고 성능을 비교하였습니다.

In [ ]:
batch_size = 32
epochs = 60

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay = 1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

batch_size = 32
epochs = 60

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

epoch : 1/60    time : 76s/4479s
TRAIN    loss : 1.72830    f1 : 0.14438
epoch : 2/60    time : 74s/4308s
TRAIN    loss : 0.83445    f1 : 0.18126
epoch : 3/60    time : 74s/4198s
TRAIN    loss : 0.63796    f1 : 0.24646
epoch : 4/60    time : 75s/4193s
TRAIN    loss : 0.52001    f1 : 0.34189
epoch : 5/60    time : 74s/4091s
TRAIN    loss : 0.44656    f1 : 0.41143
epoch : 6/60    time : 75s/4029s
TRAIN    loss : 0.39889    f1 : 0.49143
epoch : 7/60    time : 74s/3918s
TRAIN    loss : 0.34174    f1 : 0.53181
epoch : 8/60    time : 74s/3873s
TRAIN    loss : 0.30967    f1 : 0.59707
epoch : 9/60    time : 74s/3773s
TRAIN    loss : 0.26334    f1 : 0.64049
epoch : 10/60    time : 75s/3738s
TRAIN    loss : 0.24932    f1 : 0.66951
epoch : 11/60    time : 74s/3619s
TRAIN    loss : 0.20917    f1 : 0.73522
epoch : 12/60    time : 75s/3576s
TRAIN    loss : 0.19565    f1 : 0.74624
epoch : 13/60    time : 74s/3480s
TRAIN    loss : 0.18186    f1 : 0.76510
epoch : 14/60    time : 74s/3427s
TRAIN    loss

### 추론

In [ ]:
model.eval()
f_pred = []
pred_prob = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
            pred_prob.extend(pred.detach().cpu().numpy())
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

### 제출물 생성

In [ ]:
sub = pd.read_csv(path + "sample_submission.csv")

sub["label"] = f_result

sub

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


## 후처리

In [ ]:
import os
import glob
from os.path import join as opj

import numpy as np
import pandas as pd 
from tqdm import tqdm
from easydict import EasyDict
from torch.cuda.amp import autocast
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from torch.utils.data import Dataset, DataLoader

In [ ]:
train_y = pd.read_csv(path + "/train_df.csv")
train_y.head()

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good


In [ ]:
# Make train_df_bad.csv

one = [l[1][4] if l[1][3] == 'good' else l[1][2]+'-bad' for l in train_y.iterrows()]
train_y['label'] = one
train_y.to_csv(path + '/train_df_bad.csv', index=False)

In [ ]:
# Make <class>_df.csv
c = 'zipper'
train_y = pd.read_csv(path + "/train_df.csv")
train_y[train_y['class']==c].to_csv(path + f'/{c}_df.csv', index=False)

In [ ]:
bad_df = pd.read_csv(path + '/train_df_bad.csv')  
 
le_bad = LabelEncoder() #le_bad
bad_df['label'] = le_bad.fit_transform(bad_df['label'])

good = le_bad.transform([label for label in le_bad.classes_ if 'good' in label]) #30개
ngood = le_bad.transform([label for label in le_bad.classes_ if not 'good' in label])

train_df = pd.read_csv(path + '/train_df.csv')
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])
good2 = le.transform([label for label in le.classes_ if 'good' in label]) # 88개

In [ ]:
bad_df.head(n=10)

,index,file_name,class,state,label
0,0,10000.png,transistor,good,25
1,1,10001.png,capsule,good,5
2,2,10002.png,transistor,good,25
3,3,10003.png,wood,good,27
4,4,10004.png,bottle,good,1
5,5,10005.png,wood,good,27
6,6,10006.png,capsule,good,5
7,7,10007.png,screw,good,19
8,8,10008.png,cable,bent_wire,2
9,9,10009.png,transistor,good,25


In [ ]:
# 앙상블 예측 함수
def get_preds(li, good=good, ngood=ngood, good2=good2, le=le):
    ww = np.array([np.load(i) for i in li]) 
    w = ww.mean(axis=0) 
    w_maxs = np.max(w, axis=1)
    w_preds = np.argmax(w, axis=1)

    df_k2 = pd.DataFrame(data = w_maxs, columns=['max'])
    df_k2['preds'] = w_preds
    df_k2['label'] = le.inverse_transform(w_preds)

    bad2 = np.load(path + '/../files/effb4_bad_5fold.npy')

    bad2_maxs = np.max(bad2, axis=1)
    bad2_preds = np.argmax(bad2, axis=1)
    df_bad2 = pd.DataFrame(data = bad2_maxs, columns=['max'])
    df_bad2['preds'] = bad2_preds

    # good-bad에서 bad로 예측하거나 good으로 예측해도 softmax값이 0.999999보다 작은 인덱스들 추출
    idx2 = np.array(df_bad2[((df_bad2['preds'].isin(good)) & (df_bad2['max'] <0.999999)) | df_bad2['preds'].isin(ngood)].index)

    #위에서 구한 인덱스들 중에서 예측 레이블이 good인 경우면 2번째 높은 레이블로 변경
    idx_bad2 = np.array(df_k2.loc[idx2][df_k2['label'].isin(le.inverse_transform(good2))].index)
    p_bad2 = np.argsort(w, axis=1)[idx_bad2, -2]
    
    df_k2['label'].iloc[idx_bad2]= le.inverse_transform(p_bad2)
        
    return df_k2['label'].values

In [ ]:
li = glob.glob(path + '/../files/softmax_*.npy')
sub['label'] = get_preds(li, good=good, ngood=ngood, good2=good2, le=le)
sub.head()

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good


In [ ]:
def Postprocessing_oneclass(cls, sub, npys):
    df_sub = sub.copy()
    idxLst = [df_sub.iloc[idx]['index'] for idx in range(len(df_sub)) if cls in df_sub.iloc[idx]['label']]
    
    if not npys:
        raise AssertionError('npys must not be empty') 
    # 단일모델 예측 : 기존 모델 예측값 대신 단일 모델 예측값으로 전부 변경
    elif len(npys) == 1:
        path = npys[0]
        p = np.load(path, allow_pickle=True)
        df_sub.loc[idxLst,'label'] = p 

    # 하드보팅 예측 : 단일 모델들의 예측값과 원래의 예측값에 대하여 hard voting
    else:
        df = df_sub[df_sub['index'].isin(idxLst)]

        for path in npys:
            num = os.path.basename(path).split('.')[0][-3:]
            p = np.load(path, allow_pickle=True)
            df[f'pred_{num}'] = p
        
        for i in range(len(df)):
            label_pred_list = [df.iloc[i,1],df.iloc[i,2],df.iloc[i,3],df.iloc[i,4]]
            if (len(Counter(label_pred_list).most_common(2)) >1) and (Counter(label_pred_list).most_common(2)[1][1] == 2):
     
                newlabel = df_sub.loc[df.iloc[i]['index'],'label']
  
            else:
                newlabel = max(label_pred_list, key=label_pred_list.count)
            
            df_sub.loc[df.iloc[i]['index'],'label'] = newlabel
        
    return df_sub

In [ ]:
sub1 = Postprocessing_oneclass('toothbrush', sub, [path + '/../files/toothbrush_220.npy',path + '/../files/toothbrush_221.npy',path + '/../files/toothbrush_222.npy'])
sub2 = Postprocessing_oneclass('zipper', sub1, [path + '/../files/zipper_254.npy', path + '/../files/zipper_255.npy', path + '/../files/zipper_256.npy'])

sub2.to_csv(path + '../best_ensemble5.csv',index=False)
